<!-- # CNN autoencoder and Clustering from MTRX data

Use this notebook to load Scienta Omicron Matrix format SPM data and create standardised images for machine learning training and analysis. The code can generate both JPG image data, useful for manually checking the data, and windowed numpy data that can be loaded into ML models. 

The notebook then creates an autoencoder for training on a large dataset, followed by KMEANS clustering. 

**Author**: Steven R. Schofield  
**Created**: November, 2024 -->

# CASTEP - Si 001 surface
## Steven R. Schofield (Universtiy College London) May 2025

### Determine appropriate paths whether we are working on macbook or the cluster

In [44]:
import sys
from pathlib import Path

# Define candidate paths using Path objects
module_path_list = [
    Path('/Users/steven/academic-iCloud/Python/modules'),
    Path('/hpc/srs/Python/modules')
]

data_path_list = [
    Path('/Users/steven/academic-iCloud/Calculations/castep/'),
    Path('/hpc/srs/castep')
]

# Resolve actual paths
module_path = next((p for p in module_path_list if p.exists()), None)
data_path = next((p for p in data_path_list if p.exists()), None)

# Check and report missing paths
if module_path is None:
    print("Error: Could not locate a valid module path.")
if data_path is None:
    print("Error: Could not locate a valid data path.")

if module_path is None or data_path is None:
    sys.exit(1)

# Add module_path to sys.path if needed
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path))

# Print resolved paths
print(f"module_path = {module_path}")
print(f"data_path = {data_path}")

module_path = /Users/steven/academic-iCloud/Python/modules
data_path = /Users/steven/academic-iCloud/Calculations/castep


### Import modules

In [45]:
# # Ensure modules are reloaded 
%load_ext autoreload
%autoreload 2

# Import standard modules
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt

# Import custom module
import SRSCALCUTILS.castep_tools as ct

from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Paths for the calculation

In [46]:
# General
job_folder = 'work-2025/si001_surf'
job_path = data_path / job_folder
verbose = False             # Set this True to print out more information

# Print resolved paths
print(f"job_path = {job_path}")

# If it doesn’t exist, create it (including any missing parent folders)
if not os.path.isdir(job_path):
    os.makedirs(job_path, exist_ok=True)

# Now safely switch into it
os.chdir(job_path)

print('Current files in this directory:')
for file in os.listdir():
    print(file)

job_path = /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf
Current files in this directory:
si001_test.cell
castep_input.xyz
si001_test.param


### Base parameters of calculation and unit cell

In [47]:
# Set up the param and cell files for the calculation
filename = "si001_surf"
title = "silicon 001 surface tests"

params = {'task' : 'geomopt',
          'xc_functional'           : 'PBE',
          'cut_off_energy'          : 400,
          'spin_polarised'          : 'false',
          'write_cell_structure'    : 'true',
          'charge'                  : 0,
          'nextra_bands'            : 8,
          'geom_energy_tol'         : 0.00005,          # default: 0.00005 eV 
          'geom_disp_tol'           : 0.002,            # default: 0.002 Ang.
          'geom_max_iter'           : 100,              # default: 30
          'geom_convergence_win'    : 2,                # default: 2
          'max_scf_cycles'          : 300               # default: 30
          }              

lattice_cart = np.array([
            [3.8641976,     0.0,     0.0],
            [0.0,     3.8641976,     0.0],
            [0.0,     0.0,     5.4648012]
        ])

positions_frac = np.array([
            ['Si', 0.0,  0.0,  0.0],
            ['Si', 0.5,  0.0,  0.25],
            ['Si', 0.5,  0.5,  0.5],
            ['Si', 0.0,  0.5,  0.75],
        ])

cell_constraints = np.array([
            [0,  0,  0],
            [0,  0,  0]
        ])

kpoints_mp_grid = np.array([2, 2, 2])

fix_all_ions = False
symmetry_generate=False
symmetry_tol = 0.01


### Build supercell

In [ ]:
n = [2, 2, 1] 
positions_frac, lattice_cart = ct.create_supercell_from_fractional_coords(positions_frac,lattice_cart,n)
positions_frac = ct.sort_positions_frac(positions_frac)

### TESTING

In [55]:
# Convert
positions_cart = ct.frac_to_cart(lattice_cart, positions_frac)

# Write .xyz
xyz = ct.write_xyz(positions_cart, path=job_path, filename='castep_input', comment='Converted by SRSCALCUTILS')
print(positions_frac)
print()
print(positions_cart)

[['Si' 0.5 0.75 0.75]
 ['Si' 0.0 0.75 0.75]
 ['Si' 0.5 0.25 0.75]
 ['Si' 0.0 0.25 0.75]
 ['Si' 0.75 0.75 0.5]
 ['Si' 0.25 0.75 0.5]
 ['Si' 0.75 0.25 0.5]
 ['Si' 0.25 0.25 0.5]
 ['Si' 0.75 0.5 0.25]
 ['Si' 0.25 0.5 0.25]
 ['Si' 0.75 0.0 0.25]
 ['Si' 0.25 0.0 0.25]
 ['Si' 0.5 0.5 0.0]
 ['Si' 0.0 0.5 0.0]
 ['Si' 0.5 0.0 0.0]
 ['Si' 0.0 0.0 0.0]]

[['Si' 3.8641976 5.796296399999999 4.0986009]
 ['Si' 0.0 5.796296399999999 4.0986009]
 ['Si' 3.8641976 1.9320988 4.0986009]
 ['Si' 0.0 1.9320988 4.0986009]
 ['Si' 5.796296399999999 5.796296399999999 2.7324006]
 ['Si' 1.9320988 5.796296399999999 2.7324006]
 ['Si' 5.796296399999999 1.9320988 2.7324006]
 ['Si' 1.9320988 1.9320988 2.7324006]
 ['Si' 5.796296399999999 3.8641976 1.3662003]
 ['Si' 1.9320988 3.8641976 1.3662003]
 ['Si' 5.796296399999999 0.0 1.3662003]
 ['Si' 1.9320988 0.0 1.3662003]
 ['Si' 3.8641976 3.8641976 0.0]
 ['Si' 0.0 3.8641976 0.0]
 ['Si' 3.8641976 0.0 0.0]
 ['Si' 0.0 0.0 0.0]]


In [ ]:
def select_atoms_by_plane(
    positions_frac,
    lattice_cart,
    plane,
    reference_position,
    tolerance=0.0
):
    """
    Select atoms lying within a slab around a specified plane.

    Parameters
    ----------
    positions_frac : array-like of shape (N, 4)
        Rows of [element_symbol, frac_x, frac_y, frac_z].
    lattice_cart : array-like of shape (3, 3)
        Cartesian cell vectors (each row is a lattice vector in Å).
    plane : array-like of length 3
        Normal vector (in Cartesian coordinates) defining the plane.
    reference_position : array-like of length 3
        A point (in Cartesian coordinates) known to lie on the plane.
    tolerance : float, optional
        Distance tolerance in Å. Atoms whose perpendicular distance to the plane
        is ≤ tolerance are selected (default = 0.0).

    Returns
    -------
    result : list of lists, shape (M, 6)
        Each entry corresponds to one atom and contains:
        [index (1-based), is_selected (bool), element_symbol,
         frac_x, frac_y, frac_z, distance]
        where `distance` is the signed perpendicular distance (Å) from the atom
        to the plane (positive on the side of the normal vector).
    """
    # Convert inputs
    arr = np.array(positions_frac, dtype=object)
    symbols = arr[:, 0]
    frac = arr[:, 1:].astype(float)
    lattice = np.array(lattice_cart, dtype=float)

    # Compute Cartesian positions of atoms
    cart = frac.dot(lattice)

    # Plane normal and reference point
    normal = np.array(plane, dtype=float)
    if np.allclose(normal, 0):
        raise ValueError("Plane normal vector must be nonzero.")
    # unit normal
    n_unit = normal / np.linalg.norm(normal)
    ref_pt = np.array(reference_position, dtype=float)

    # Compute signed distances
    # For each atom: d = dot(r_i - ref_pt, n_unit)
    deltas = cart - ref_pt
    dists = deltas.dot(n_unit)

    # Build result list
    result = []
    for i, (sym, fcoord, dist) in enumerate(zip(symbols, frac, dists)):
        sel = abs(dist) <= tolerance
        result.append([
            i + 1,
            bool(sel),
            sym,
            float(fcoord[0]),
            float(fcoord[1]),
            float(fcoord[2]),
            float(dist)
        ])

    return result


In [64]:
# First select the atoms on the bottom layer 
axis = 'z'
criteria = "maximum"
ref_value, _ = ct.find_plane_value(positions_frac, lattice_cart, axis, criteria)
tolerance = 0.5

plane = (0,0,1)
reference_position = (3.8641976, 5.796296399999999, 4.0986009)

labelled_positions_frac = select_atoms_by_plane(
    positions_frac,
    lattice_cart,
    plane,
    reference_position,
    tolerance=tolerance
)

In [74]:
(a,b) = ct.extract_plane_lattice_vectors(
    labelled_positions_frac,
    lattice_cart
)

array([ 0.       , -3.8641976,  0.       ])

### Add hydrogen termination to cell bottom 

In [ ]:
# # First select the atoms on the bottom layer 
# axis = 'z'
# criteria = "minimum"
# ref_value, _ = ct.find_plane_value(positions_frac, lattice_cart, axis, criteria)
# tolerance = 0.5
# labelled_positions_frac = ct.select_atoms_by_plane(positions_frac, lattice_cart, axis, ref_value,
#                           tolerance=tolerance, include=None, exclude=None)

# # Next calculate the relative bond vectors for the two hydrogen atoms (per Si)
# phi = np.deg2rad(90)    
# si_h_bond_length = 1.5                         
# theta_h_1 = np.deg2rad(180) - np.arccos(-1/3) / 2                
# theta_h_2 = np.deg2rad(180) + np.arccos(-1/3) / 2            
# bond_h_1  = ct.bond_vector_from_spherical(theta_h_1, phi, si_h_bond_length)                   
# bond_h_2  = ct.bond_vector_from_spherical(theta_h_2, phi, si_h_bond_length)

# # Now add the hydrogen atoms to the silicon atoms selected above using the calculated bond vectors
# positions_frac, lattice_cart = ct.add_atoms_to_positions_frac(
#                                     labelled_positions_frac,
#                                     lattice_cart,
#                                     [bond_h_1,bond_h_2],
#                                     extend_unit_cell=(0, 0, 1),
#                                     atom="H")
# positions_frac = ct.sort_positions_frac(positions_frac)

### Dimerise surface

In [7]:
# # First select the atoms on the bottom layer 
# axis = 'z'
# criteria = "maximum"
# ref_value, _ = ct.find_plane_value(positions_frac, lattice_cart, axis, criteria)
# tolerance = 0.5
# labelled_positions_frac = ct.select_atoms_by_plane(positions_frac, lattice_cart, axis, ref_value,
#                           tolerance=tolerance, include=None, exclude=None)

# # Dimerise surface
# dimer_direction = "y"
# displacement_direction = "y"
# displacement = 0.6
# start_phase='+'
# alternate=False
# positions_frac = dimerise_displacement(labelled_positions_frac,
#     lattice_cart,
#     dimer_direction = dimer_direction,
#     displacement_direction = displacement_direction,
#     displacement = displacement,
#     start_phase=start_phase,
#     alternate=alternate,
#     wrap_axes=(1,1,1)
# )

# # labelled_positions_frac = ct.select_atoms_by_plane(positions_frac, lattice_cart, axis, ref_value,
# #                           tolerance=tolerance, include=None, exclude=None)

# # displacement = 0.3
# # displacement_direction = "z"
# # alternate=True
# # positions_frac = dimerise_displacement(labelled_positions_frac,
# #     lattice_cart,
# #     dimer_direction = dimer_direction,
# #     displacement_direction = displacement_direction,
# #     displacement = displacement,
# #     start_phase=start_phase,
# #     alternate=alternate,
# #     wrap_axes=(1,1,1)
# # )

[[1, False, 'Si', 0.5, 0.75, 0.75, 3.6200908246],
 [2, False, 'Si', 0.0, 0.75, 0.75, 3.6200908246],
 [3, False, 'Si', 0.5, 0.25, 0.75, 3.6200908246],
 [4, False, 'Si', 0.0, 0.25, 0.75, 3.6200908246],
 [5, False, 'Si', 0.75, 0.75, 0.5, 2.2538905246],
 [6, False, 'Si', 0.25, 0.75, 0.5, 2.2538905246],
 [7, False, 'Si', 0.75, 0.25, 0.5, 2.2538905246],
 [8, False, 'Si', 0.25, 0.25, 0.5, 2.2538905246],
 [9, False, 'Si', 0.75, 0.5, 0.25, 0.8876902246],
 [10, False, 'Si', 0.25, 0.5, 0.25, 0.8876902246],
 [11, False, 'Si', 0.75, 0.0, 0.25, 0.8876902246],
 [12, False, 'Si', 0.25, 0.0, 0.25, 0.8876902246],
 [13, True, 'Si', 0.5, 0.5, 0.0, -0.4785100754],
 [14, True, 'Si', 0.0, 0.5, 0.0, -0.4785100754],
 [15, True, 'Si', 0.5, 0.0, 0.0, -0.4785100754],
 [16, True, 'Si', 0.0, 0.0, 0.0, -0.4785100754]]

### Add hydrogen to the top

In [ ]:
# # First select the atoms on the bottom layer 
# axis = 'z'
# criteria = "maximum"
# ref_value, _ = ct.find_plane_value(positions_frac, lattice_cart, axis, criteria)
# tolerance = 0.5
# labelled_positions_frac = ct.select_atoms_by_plane(positions_frac, lattice_cart, axis, ref_value,
#                           tolerance=tolerance, include=None, exclude=None)

# # Next calculate the relative bond vectors for the two hydrogen atoms (per Si)
# phi = np.deg2rad(90)    
# si_h_bond_length = 1.5                         
# theta_h_1 = np.arccos(-1/3) / 2                
# theta_h_2 = np.arccos(-1/3) / 2            
# bond_h_1  = ct.bond_vector_from_spherical(theta_h_1, phi, si_h_bond_length)                   
# bond_h_2  = ct.bond_vector_from_spherical(theta_h_2, phi, si_h_bond_length)

# # Now add the hydrogen atoms to the silicon atoms selected above using the calculated bond vectors
# positions_frac, lattice_cart = ct.add_atoms_to_positions_frac(
#                                     labelled_positions_frac,
#                                     lattice_cart,
#                                     [bond_h_1,bond_h_2],
#                                     extend_unit_cell=(0, 0, 1),
#                                     atom="H")
# positions_frac = ct.sort_positions_frac(positions_frac)

### Add vacuum spacing

In [ ]:
# vac = 10
# positions_frac, lattice_cart = ct.create_vacuum_spacing(positions_frac, lattice_cart, vac)

### Add ionic contraints

In [ ]:
# # Create fixed ion constraints
# conditions = "z < 2.5"
# ionic_constraints = ct.select_atoms_by_region(positions_frac, lattice_cart, conditions)

In [13]:
filename = f"si001_test"
castep_path = job_path / f"{filename}.castep"

print('=' * 80 + f'\nCASTEP file: {castep_path}\n' + '=' * 80 + '\n')   

param_filename = ct.write_param_file(
    params,
    title = title,
    filename=filename,
    path=job_path,
    display_file=True
    )

cell_filename = ct.write_cell_file(
    title = title,
    path=job_path,
    filename=filename,
    lattice_cart=lattice_cart,
    positions_frac=positions_frac,
    cell_constraints=cell_constraints,
    ionic_constraints=ionic_constraints,
    fix_all_ions=fix_all_ions,
    symmetry_generate=symmetry_generate,
    symmetry_tol = symmetry_tol,
    kpoints_mp_grid=kpoints_mp_grid,
    display_file=True
)

CASTEP file: /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf/si001_test.castep

Wrote param file to: /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf/si001_test.param
!TITLE: silicon 001 surface tests

TASK                 : geomopt
XC_FUNCTIONAL        : PBE
CUT_OFF_ENERGY       : 400
SPIN_POLARISED       : false
WRITE_CELL_STRUCTURE : true
CHARGE               : 0
NEXTRA_BANDS         : 8
GEOM_ENERGY_TOL      : 5e-05
GEOM_DISP_TOL        : 0.002
GEOM_MAX_ITER        : 100
GEOM_CONVERGENCE_WIN : 2
MAX_SCF_CYCLES       : 300


Wrote cell file to: /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf/si001_test.cell
! silicon 001 surface tests

%BLOCK lattice_cart
   ANG
       7.7283952000    0.0000000000    0.0000000000
       0.0000000000    7.7283952000    0.0000000000
       0.0000000000    0.0000000000   21.7956278038
%ENDBLOCK lattice_cart

%BLOCK CELL_CONSTRAINTS
       0    0    0
       0    0    0
%ENDBLOCK CELL_CO

In [14]:
# if str(module_path) == '/hpc/srs/Python/modules':
#     !bash -l -c "mpirun -np 62 castep.mpi {filename}"
# else:
#     !zsh -l -c "castepmpi {filename}"


In [15]:

# unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)
# energy_optimisation = ct.get_LBFGS_energies(castep_path)
# energies = [val for _, val in energy_optimisation]
# energy = energies[-1]

# print('Optimised energy {} eV'.format(energy))
# print('Optimised lattice constants a,b = {} Ang., c = {} Ang.'.format(a,c))
# print('Unit cell:')
# for line in unit_cell:
#     print(' '*2,np.abs(line))
